In [137]:
import numpy as np 
import os
import matplotlib
from PIL import Image, ImageOps
import random
import cv2
import csv

In [47]:
folder = r"D:\Datasets\ND-Contact-Lens\LG4000\images"
image_name = "04261d1016" + ".tiff"

image_path = os.path.join(folder, image_name)
print(image_path)

D:\Datasets\ND-Contact-Lens\LG4000\images\04261d1016.tiff


In [55]:
def set_img_name(origin_path, destination_path, i = ""):
    file_name = os.path.basename(origin_path)
    file_name = file_name.split(".")
    file_name = file_name[0] + "_" + str(i) + "." + file_name[1]
    destination_path = os.path.join(destination_path, file_name)
    return destination_path

def random_translation(width, height):
    limits_x = int(width * .75 ) // 2
    limits_y = int(height * .65 ) // 2

    x = random.randint(-limits_x, limits_x)
    y = random.randint(-limits_y, limits_y)

    return x, y

In [135]:
def translate_img(image, width, height):
    new_image = Image.new("RGB", (width, height))
    
    #translating the image
    displacement_x, displacement_y = random_translation(width, height)
    paste_position = (displacement_x, displacement_y)
    new_image.paste(image, paste_position)

    return new_image, displacement_x, displacement_y

def mirror_image(image, width, height):
    new_image, displacement_x, displacement_y = translate_img(image, width, height)
    new_image = ImageOps.mirror(new_image)
    displacement_x = (-displacement_x)
    return new_image, displacement_x, displacement_y

def zoom_and_crop(image):
    # Making a random zoom factor and keeping it between the range (1,2]
    zoom_factor = 2 - random.random()
    
    # Calculate the new dimensions after zooming
    width, height = image.size
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)

    # zoom image
    zoomed_image = image.resize((new_width, new_height))

    # Calculate the coordinates for cropping
    left = (new_width - width) / 2
    top = (new_height - height) / 2
    right = (new_width + width) / 2
    bottom = (new_height + height) / 2

    # Crop the image to the original resolution
    cropped_image = zoomed_image.crop((left, top, right, bottom))

    return cropped_image, zoom_factor

def save_img(image, image_path, new_destination, i):
    image_name = set_img_name(image_path, new_destination, i)
    image.save(image_name, "TIFF")

    return image_name.split("/")[-1]

In [136]:
def data_augmentation(image_path, destination_folder = "./"):
    original_image = Image.open(image_path)
    width,height = original_image.size 

    imgs = []


    #Translating image
    new_image, displacement_x, displacement_y = translate_img(original_image, width, height)
    new_image_path = save_img(new_image, image_path, destination_folder, 0)
    imgs.append({"path": new_image_path, "displacement_x": displacement_x, "displacement_y": displacement_y, "zoom_factor": 1})

    # Mirroring one image
    new_image, displacement_x, displacement_y = mirror_image(original_image, width, height)
    new_image_path = save_img(new_image, image_path, destination_folder, 1)
    imgs.append({"path": new_image_path, "displacement_x": displacement_x, "displacement_y": displacement_y, "zoom_factor": 1})

    #Zooming image
    new_image, zoom_factor = zoom_and_crop(original_image)
    new_image_path = save_img(new_image, image_path, destination_folder, 2)
    imgs.append({"path": new_image_path, "displacement_x": 0, "displacement_y": 0, "zoom_factor": zoom_factor})

    return imgs


imgs = data_augmentation(image_path)
for img in imgs:
    print(img)

{'path': '04261d1016_0.tiff', 'displacement_x': -154, 'displacement_y': 106, 'zoom_factor': 1}
{'path': '04261d1016_1.tiff', 'displacement_x': -49, 'displacement_y': 106, 'zoom_factor': 1}
{'path': '04261d1016_2.tiff', 'displacement_x': 0, 'displacement_y': 0, 'zoom_factor': 1.2099813773601462}


In [141]:
def read_dataset(dataset = "LG4000", train_verification = "train"):
    assert dataset == "LG4000" or dataset == "AD100"
    assert train_verification == "train" or train_verification == "verification"

    train_verification = "verification-subject-disjoint" if train_verification == "verification" else "train"

    data = []

    csvreader = None
    with open(r"D:\Datasets\ND-Contact-Lens\LG4000\train.csv", "r") as file:
        csvreader = list(csv.reader(file))

    csvreader.pop(0) # Removing the header line of the file

    for row in csvreader:
        data.append({"image": row[0], "x": int(row[6]), "y":int(row[7]), "radius": int(row[8])})

    del csvreader

    return data

dataset = read_dataset()

In [143]:
print(len(dataset))

3000


In [ ]:
#Chamar o data augmentation para cada imagem

#Salvar os novos valores em um novo csv